In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/.julia/dev/StableManipulation/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Plots
using Convex, SCS
using JLD
using StableManipulation

In [3]:
include("../models/box_ground_frictional.jl")

Main.BoxWorld

In [4]:
dynamics! = BoxWorld.ode_dynamics!
conditions = BoxWorld.ode_conditions
affect! = BoxWorld.ode_affect!
affect_neg! = BoxWorld.ode_affect_neg!

domain = BoxWorld.domain
guard_set = BoxWorld.guard_set
jumpmap = BoxWorld.jumpmap

n_contacts = BoxWorld.n_contacts
Δt = BoxWorld.Δt
modes = BoxWorld.modes

tol_c = BoxWorld.tol_c

1.0e-5

In [5]:
xref = [0; BoxWorld.h/2; 0; 1; 0; 0] # sliding to the right at velocity of 1
uref = [BoxWorld.μ*BoxWorld.m*BoxWorld.g; 0; 0] # reference body wrench
nominal_mode = [1 1 1 1] # right sliding mode

pusher_p = [-BoxWorld.w/2;0] # pusher_location

function pusher_box_discrete_dynamics(x, pusher_u, mode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = Jc*pusher_u
    xn = BoxWorld.discrete_dynamics(x, u, mode)
    return xn
end

function pusher_box_dynamics(x, pusher_u, mode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = Jc*pusher_u
    dx = BoxWorld.continuous_dynamics_differentiable(x, u, mode)
    return dx
end

pusher_u_ref = [uref[1]; 0]

2-element Vector{Float64}:
 4.905
 0.0

In [6]:
# Linearize over xref and uref
A = ForwardDiff.jacobian(_x->pusher_box_dynamics(_x, pusher_u_ref, nominal_mode), xref)
B = ForwardDiff.jacobian(_u->pusher_box_dynamics(xref, _u, nominal_mode), pusher_u_ref)


6×2 Matrix{Float64}:
  0.0   0.0
  0.0   0.0
  0.0   0.0
  1.0   0.5
  0.0  -5.0e-7
 -0.0   6.0e-7

In [7]:
A

6×6 Matrix{Float64}:
 0.0  0.0   0.0         1.0  0.0  0.0
 0.0  0.0   0.0         0.0  1.0  0.0
 0.0  0.0   0.0         0.0  0.0  1.0
 0.0  0.0   7.84801e-7  0.0  0.0  0.0
 0.0  0.0  -1.5696e-6   0.0  0.0  0.0
 0.0  0.0  -9.81001e-6  0.0  0.0  0.0

In [17]:
B

6×2 Matrix{Float64}:
  0.0   0.0
  0.0   0.0
  0.0   0.0
  1.0   0.5
  0.0  -5.0e-7
 -0.0   6.0e-7

In [18]:
Q = Semidefinite(6)
Y = Variable(2,6)

Variable
size: (2, 6)
sign: real
vexity: affine
id: 397…306

In [19]:
prob = maximize(sum(Q) + tr(Q))


maximize
└─ + (affine; real)
   ├─ sum (affine; real)
   │  └─ 6×6 real variable (id: 181…552)
   └─ sum (affine; real)
      └─ diag (affine; real)
         └─ …

status: `solve!` not called yet

In [20]:
prob.constraints += isposdef( -(A*Q + Q'*A' + B*Y + Y'*B'))

1-element Vector{Constraint}:
 sdp constraint (affine)
└─ - (affine; real)
   └─ + (affine; real)
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      └─ adjoint (affine; real)
         └─ …

In [22]:
Convex.solve!(prob,() -> SCS.Optimizer(verbose=true))

----------------------------------------------------------------------------
	SCS v2.1.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 202
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 49, constraints m = 73
Cones:	primal zero / dual free vars: 31
	sd vars: 42, sd blks: 2
Setup time: 1.06e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.86e+13  7.17e+19  1.00e+00 -1.28e+14 -0.00e+00  8.60e+13  1.25e-04 
   100| 7.30e+09  2.31e+16  1.00e+00 -5.45e+13 -0.00e+00  5.45e+13  2.31e-03 
   200| 6.52e+09  1.52e+16  1.00e+00 -5.47e+13 -0.00e+00  5.47e+13  4.32e-03 
   300| 1

┌ Warning: Problem status DUAL_INFEASIBLE; solution may be inaccurate.
└ @ Convex /home/xianyi/.julia/packages/Convex/uI27T/src/solution.jl:263


In [23]:
Q_sol = evaluate(Q)

6×6 Matrix{Float64}:
  0.350083   -0.112388     0.0236643   -0.176596    0.058528    -0.0787655
 -0.112388    0.217759    -1.02812e-5  -0.0194054  -0.00229415   5.80947e-5
  0.0236643  -1.02878e-5   0.00490562  -0.0275449   0.0120987   -0.016338
 -0.176595   -0.019404    -0.0275451    0.198267   -0.0678231    0.091786
  0.0585283  -0.00229165   0.0120985   -0.0678234   0.0299012   -0.0403223
 -0.078766    5.75904e-5  -0.0163378    0.0917858  -0.0403227    0.0544362

In [24]:
eigvals(Q_sol)

6-element Vector{Float64}:
 -1.2455214375552821e-6
  4.226435377953292e-6
  0.010955919729507812
  0.06038735016164899
  0.24624384482578232
  0.5377619745902628

In [25]:
Y_sol = evaluate(Y)

2×6 Matrix{Float64}:
 -0.102956  -0.155429   21.438   -0.0264712  -17.3245  -6.1274
 -0.181396   0.447601  -43.0586  -0.0528608   34.1987  12.8614

In [39]:
Q_sol[abs.(Q_sol) .< 1e-4] .= 0
P = inv(Q_sol)

6×6 Matrix{Float64}:
    -5.70939    -20.6777  -2493.2        …  -1611.15       -1935.3
   -20.683      -38.5769  -4290.79          -2859.85       -3390.72
 -2493.99     -4291.14    94413.7              -2.56144e5     -1.5917e5
    -8.69282    -26.8825  -3452.09          -2220.27       -2711.62
 -1609.98     -2857.2        -2.55938e5        -1.73744e5     -2.04098e5
 -1934.69     -3388.91       -1.59033e5  …     -2.04162e5     -1.97164e5

In [43]:
P_bar = P/maximum(abs.(P))

6×6 Matrix{Float64}:
 -2.22898e-5  -8.07269e-5   -0.00973359  …  -0.00629004  -0.00755551
 -8.07478e-5  -0.000150607  -0.0167515      -0.011165    -0.0132376
 -0.0097367   -0.0167529     0.368597       -1.0         -0.621408
 -3.39373e-5  -0.000104951  -0.0134772      -0.00866807  -0.0105863
 -0.00628548  -0.0111547    -0.999197       -0.678308    -0.796812
 -0.00755315  -0.0132305    -0.620873    …  -0.797059    -0.769738

In [44]:
P_bar[abs.(P_bar) .< 1e-4] .= 0

6-element view(::Vector{Float64}, [1, 2, 4, 7, 19, 22]) with eltype Float64:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [55]:
P_bar[:,1:3]

6×3 Matrix{Float64}:
  0.0          0.0          -0.00973359
  0.0         -0.000150607  -0.0167515
 -0.0097367   -0.0167529     0.368597
  0.0         -0.000104951  -0.0134772
 -0.00628548  -0.0111547    -0.999197
 -0.00755315  -0.0132305    -0.620873

In [54]:
P_bar[:,4:6]

6×3 view(::Matrix{Float64}, :, 4:6) with eltype Float64:
  0.0          -0.00629004  -0.00755551
 -0.000105062  -0.011165    -0.0132376
 -0.0135098    -1.0         -0.621408
  0.0          -0.00866807  -0.0105863
 -0.0086671    -0.678308    -0.796812
 -0.0105955    -0.797059    -0.769738

In [58]:
P_bar # why P_bar is like this???

6×6 Matrix{Float64}:
 0.0  0.0   0.0       0.0   0.0        0.0
 0.0  0.0   0.0       0.0   0.0        0.0
 0.0  0.0   0.368597  0.0  -1.0       -0.621408
 0.0  0.0   0.0       0.0   0.0        0.0
 0.0  0.0  -0.999197  0.0  -0.678308  -0.796812
 0.0  0.0  -0.620873  0.0  -0.797059  -0.769738